In [2]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

file_name = 'reviews_complete.csv'

# Se carga del Dataset
df = pd.read_csv(file_name)

# Se cuentan los valores absolutos de la columna 'polarity'
class_counts = df['polarity'].value_counts()
print("Conteo de valores (Absoluto):")
print(class_counts)
        
# Se analiza porcentualmente para ver si hay desbalanceo o no
class_percentages = df['polarity'].value_counts(normalize=True) * 100
print("Distribución de clases (Porcentaje %):")
print(class_percentages)

Conteo de valores (Absoluto):
polarity
Recommended        8179
Mixed Feelings     1005
Not Recommended     619
Name: count, dtype: int64
Distribución de clases (Porcentaje %):
polarity
Recommended        83.433643
Mixed Feelings     10.251964
Not Recommended     6.314394
Name: proportion, dtype: float64


In [3]:
# Se define el mapeo deseado para representar la polaridad positiva, negativa y neutra.
label_map = {
    'Recommended': 1,
    'Mixed Feelings': 0,
    'Not Recommended': -1
}

# Se aplica el mapeo
df['polarity_encoded'] = df['polarity'].map(label_map)

# Se muestra si todas las clases fueron mapeadas 
print(df['polarity_encoded'].value_counts(dropna=False))

# Se limpia el texto aplicando modificaciones al mismo
def clean_text_modified(text):
    # Se convierte a minusculas
    text = text.lower()
    # Se eliminan saltos de línea y tabulaciones
    text = re.sub(r'[\r\n\t]+', ' ', text) 
    # Se eliminan signos de puntuación, números y caracteres especiales
    text = re.sub(r'[^a-z\s]', '', text)
    # Se eliminan espacios en blanco extra
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Se aplica la función para crear la nueva columna 'review_cleaned'
df['review_cleaned'] = df['review_text'].apply(clean_text_modified)

# Se muestran las columnas originales y las nuevas columnas procesadas
df.head()

polarity_encoded
 1    8179
 0    1005
-1     619
Name: count, dtype: int64


,tittle,review_text,polarity,polarity_encoded,review_cleaned
0,Sousou_no_Frieren,I feel so catered to.\n\r\nIt feels like an et...,Recommended,1,i feel so catered to it feels like an eternity...
1,Sousou_no_Frieren,Time is a precious thing. The most valuable re...,Recommended,1,time is a precious thing the most valuable res...
2,Sousou_no_Frieren,Frieren: Beyond Journey's End has concluded an...,Recommended,1,frieren beyond journeys end has concluded and ...
3,Sousou_no_Frieren,The show is rated 9.37/10 on MAL at the time I...,Mixed Feelings,0,the show is rated on mal at the time i am writ...
4,Sousou_no_Frieren,There are so many words I could use on how Sou...,Recommended,1,there are so many words i could use on how sou...


In [4]:
# Se recoge la columna con el texto limpio 
X_networks = df['review_cleaned']
# Se recoge la columna con el texto sin limpiar para utilizarla en el transformer
X_transformer = df['review_text'] 
# Se recoge el valor numerico de la polaridad, es decir la columna objetivo
y = df['polarity_encoded']

# Tamaño de la separacion para las fases de entrenar y testear los modelos 80/20
TEST_SIZE = 0.2
# Semilla necesaria para mantener los resultados
RANDOM_STATE = 42

# Se divide en Train/Test para los modelos con redes y el modelo de transformer
X_train_networks, X_test_networks, X_train_transformer, X_test_transformer, y_train, y_test = train_test_split(
    X_networks,
    X_transformer,
    y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y
)

# Se comprueban los tamaños de los datasets
print(f"Tamaño del dataset de entrenamiento: {len(X_train_networks)}")
print(f"Tamaño del dataset de test: {len(X_test_networks)}")
print(f"Tamaño del dataset de entrenamiento: {len(y_train)}")
print(f"Tamaño del dataset de entrenamiento : {len(y_test)}")

Tamaño del dataset de entrenamiento: 7842
Tamaño del dataset de test: 1961
Tamaño del dataset de entrenamiento: 7842
Tamaño del dataset de entrenamiento : 1961


In [5]:
from collections import Counter

# Se calcula el número de palabras (tokens) en cada reseña limpia para escoger los valores que usaremos como limites para crear el tokenizador
df['longitud_reseña'] = df['review_cleaned'].apply(lambda x: len(x.split()))

print(df['longitud_reseña'].describe())

count     9803.000000
mean       517.219831
std        530.065524
min         20.000000
25%        185.000000
50%        348.000000
75%        663.500000
max      11472.000000
Name: longitud_reseña, dtype: float64


In [6]:
# Se define el tamaño del vocabulario y la longitud máxima de las secuencias en base al analisis previo
VOCAB_SIZE = 15000 
MAX_LENGTH = 512 
# Token para palabras fuera del vocabulario
OOV_TOKEN = "<unk>" 

# Se inicializa y se entrena el Tokenizer unicamente de la parte de train para evir overfitting
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOKEN)
tokenizer.fit_on_texts(X_train_networks)

# Se convierten los textos a secuencias de numeros
X_train_seq = tokenizer.texts_to_sequences(X_train_networks)
X_test_seq = tokenizer.texts_to_sequences(X_test_networks)

# Se aplica el padding para que todas las secuencias midan lo mismo añadiendo 0s o cortando la reseña si es mas grande del limite.
X_train_padded = pad_sequences(
    X_train_seq,
    maxlen=MAX_LENGTH,
    padding='post',    
    truncating='post'  
)

X_test_padded = pad_sequences(
    X_test_seq,
    maxlen=MAX_LENGTH,
    padding='post',
    truncating='post'
)

# Se comprueba el tamaño para asegurarnos de que esta correcto
print(f"Tamaño X_train_padded: {X_train_padded.shape}")
print(f"Tamaño X_test_padded:  {X_test_padded.shape}")

Tamaño X_train_padded: (7842, 512)
Tamaño X_test_padded:  (1961, 512)


In [7]:
# He comprobado diferentes reseñas para ver como ha ido el padding
print(f"Muestra de secuencia original al inicio: {X_train_seq[1][:15]}...")
print(f"Muestra de secuencia con padding al inicio: {X_train_padded[1][:15]}...")
print(f"Muestra de secuencia original al final: {X_train_seq[1][-15:]}...")
print(f"Muestra de secuencia con padding al final: {X_train_padded[1][-15:]}...")

Muestra de secuencia original al inicio: [3125, 61, 2140, 68, 168, 25, 32, 4, 49, 3275, 421, 9, 107, 55, 2]...
Muestra de secuencia con padding al inicio: [3125   61 2140   68  168   25   32    4   49 3275  421    9  107   55
    2]...
Muestra de secuencia original al final: [304, 136, 151, 659, 4, 99, 33, 50, 10, 107, 16, 153, 43, 12, 21]...
Muestra de secuencia con padding al final: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]...


In [17]:
# Se guardan los arrays para usarlos en google collab
np.save('Networks_X_train.npy', X_train_padded)
np.save('Networks_X_test.npy', X_test_padded)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)

print(f"Archivos .npy guardados: Networks_X_train.npy, Networks_X_test.npy, y_train.npy, y_test.npy")

# Se guardan los textos originales (sin limpieza intensa) para el Transformer
X_train_transformer.to_csv('Transformer_X_train.csv', index=False, header=['review_text'])
X_test_transformer.to_csv('Transformer_X_test.csv', index=False, header=['review_text'])

print(f"Archivos CSV guardados: Transformer_X_train.csv, Transformer_X_test.csv")

# Se guarda el tokenizer para usarlo en google collab
with open('keras_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

print(f"Objeto Tokenizer guardado: keras_tokenizer.pickle")

Archivos .npy guardados: Networks_X_train.npy, Networks_X_test.npy, y_train.npy, y_test.npy
Archivos CSV guardados: Transformer_X_train.csv, Transformer_X_test.csv
Objeto Tokenizer guardado: keras_tokenizer.pickle
